In [2]:
import cv2
import time
import numpy as np

In [198]:
####UNCOMMENT TO ASSESS COLOLR RANGE FOR INVISIBILITY CLOAK####
'''
# Add a preprocessing step to learn the color of the cloth you have
# If you have an image already to analyse the color, proceed to next step. If you have a video, grab one or 
# a few frames and save it to disk for further analysis
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture('output.avi')
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
# Read until video is completed
while(cap.isOpened()):
# Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
 
        # Display the resulting frame
        cv2.imshow('Frame',frame)
        
        # Filename
        filename = 'savedImageForColorAnalysis2.jpg'
  
        # Using cv2.imwrite() method
        # Saving the image
        cv2.imwrite(filename, frame)
 
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
 
  # Break the loop
    else: 
        break
 # When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()
'''

In [210]:
####UNCOMMENT TO ASSESS COLOLR RANGE FOR INVISIBILITY CLOAK####


# Read the saved image and assess the color range of the cloth used

clothImage = cv2.imread('savedImageForColorAnalysis2.jpg')

# Select ROI
r = cv2.selectROI("select the area", clothImage)
 
# Crop image
cropped_image = clothImage[int(r[1]):int(r[1]+r[3]), 
                      int(r[0]):int(r[0]+r[2])]

cv2.waitKey(0)
# Closes all the frames
cv2.destroyAllWindows()


In [150]:
####UNCOMMENT TO ASSESS COLOLR RANGE FOR INVISIBILITY CLOAK####
'''
# Display cropped image
cv2.imshow("Cropped image", cropped_image)
cv2.waitKey(0)
# Closes all the frames
cv2.destroyAllWindows()
'''

In [211]:
####UNCOMMENT TO ASSESS COLOLR RANGE FOR INVISIBILITY CLOAK####

# Calculate the range of the color in the cloth ROI
## Convert the color space from BGR to HSV
clothhsv = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)

clothhsvarr= np.array(clothhsv)

# Calculate the mean and standard deviation of the color hsv values for the given cloth
#mean hue
clothHueMean = np.mean(clothhsvarr[:,:,0])

#mean saturation
clothSatMean = np.mean(clothhsvarr[:,:,1])

#mean value
clothValMean = np.mean(clothhsvarr[:,:,2])

#dev hue
clothHueStd = np.std(clothhsvarr[:,:,0])

#dev saturation
clothSatStd = np.std(clothhsvarr[:,:,1])

#dev value
clothValStd = np.std(clothhsvarr[:,:,2])


# Calculate the upper and lower limit based on 2 standard deviation around mean

lowerColLim = (np.array([clothHueMean- 2*clothHueStd, clothSatMean-2*clothSatStd, clothValMean - 2*clothValStd]))\
                .astype(np.int64)

upperColLim = (np.array([clothHueMean + 2*clothHueStd, clothSatMean + 2*clothSatStd, clothValMean + 2*clothValStd]))\
                .astype(np.int64)

print(clothHueMean)

print(clothHueStd)



96.7671568627451
86.65199555170695


In [220]:
# Change color limits as per your cloak requirements
#lowerColLim = np.array([154, 50, 50])
#upperColLim = np.array([184, 255, 255])

lowerColLim = np.array([150, 120,   10])
upperColLim = np.array([224, 255, 251])

#lowerColLim = np.array([0, 0,   0])
#upperColLim = np.array([0, 0, 0])

In [221]:
##reading from the webcam
cap = cv2.VideoCapture(0)

## Allow the system to sleep for 3 seconds before the webcam starts
time.sleep(3)
count = 0
background = 0

## Capture the background in range of 60
for i in range(60):
    ret, background = cap.read()
background = np.flip(background, axis=1)

In [222]:
## Preparation for writing the ouput video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

In [223]:
## Read every frame from the webcam, until the camera is open
while (cap.isOpened()):
    ret, img = cap.read()
    if not ret:
        break
    count += 1
    
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    img = np.flip(img, axis=1)

    ## Convert the color space from BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    ## Generate masks to detect red color- change the color to fit the cloth I have
    '''
    lower_red = np.array([0, 120, 50])
    upper_red = np.array([10, 255,255])
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    lower_red = np.array([170, 120, 70])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red, upper_red)

    mask1 = mask1 + mask2
    '''
    mask1 = cv2.inRange(hsv, lowerColLim, upperColLim)
    
    ## Open and Dilate the mask image
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8))

    ## Create an inverted mask to segment out the red color from the frame
    mask2 = cv2.bitwise_not(mask1)

    ## Segment the red color part out of the frame using bitwise and with the inverted mask
    res1 = cv2.bitwise_and(img, img, mask=mask2)

    ## Create image showing static background frame pixels only for the masked region
    res2 = cv2.bitwise_and(background, background, mask=mask1)

    ## Generating the final output and writing
    finalOutput = cv2.addWeighted(res1, 1, res2, 1, 0)
    out.write(finalOutput)
    cv2.imshow("magic", finalOutput)
    cv2.waitKey(1)



cap.release()
out.release()
cv2.destroyAllWindows()

In [186]:
#cap.release()
#out.release()
#cv2.destroyAllWindows()